In [1]:
import numpy as np
import pandas as pd
import pydicom
import os
from os import listdir

from os.path import isfile, join
import matplotlib.pyplot as plt
import collections
from tqdm import tqdm_notebook as tqdm
from datetime import datetime

from math import ceil, floor, log
import cv2

import tensorflow as tf
import keras

import sys

# from keras_applications.resnet import ResNet50
from keras_applications.inception_v3 import InceptionV3

from sklearn.model_selection import ShuffleSplit
test_images_dir = '/home/sichun1247/Sampled_Test/'
train_images_dir = '/home/sichun1247/Sampled_Train/'

/home/sichun1247/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def read_testset(filename="/home/sichun1247/test_sample.csv"):
    df = pd.read_csv(filename)

    
    df = df.loc[:, ["Label", "Type", "ID_Image"]]
    df = df.set_index(['ID_Image', 'Type']).unstack(level=-1)
    
    return df

def read_trainset(filename="/home/sichun1247/train_sample.csv"):
    df = pd.read_csv(filename)

    
    
    df = df.loc[:, ["Label", "Type", "ID_Image"]]
    df = df.set_index(['ID_Image', 'Type']).unstack(level=-1)
    
    return df

train_df = read_trainset()   
test_df = read_testset()

In [3]:
import os
dirct = os.listdir('/home/sichun1247/Sampled_Train')
train_ID = []
for f_name in dirct:
    if f_name.endswith('.dcm'):
        f, _ = f_name.split('.')
        train_ID.append(f)

In [4]:
import os
dirct = os.listdir('/home/sichun1247/Sampled_Test')
test_ID = []
for f_name in dirct:
    if f_name.endswith('.dcm'):
        f, _ = f_name.split('.')
        test_ID.append(f)

In [5]:
tt = np.load('train1.npy')
pp = np.load('train2.npy')
p_train = []
for item in tt:
    p_train.append(item)
for item in pp:
    p_train.append(item)
test = np.load('test.npy')
p_test = []
for item in test:
    p_test.append(item)
l_train =  list(zip(train_ID, p_train))
l_test =  list(zip(test_ID, p_test))

In [6]:
class PredictionCheckpoint(keras.callbacks.Callback):
    
    def __init__(self, test_df, valid_df, 
                 test_images_dir=test_images_dir, 
                 valid_images_dir=train_images_dir, 
                 batch_size=32, input_size=(224, 224, 3)):
        
        self.test_df = test_df
        self.valid_df = valid_df
        self.test_images_dir = test_images_dir
        self.valid_images_dir = valid_images_dir
        self.batch_size = batch_size
        self.input_size = input_size
        
    def on_train_begin(self, logs={}):
        self.test_predictions = []
        self.valid_predictions = []
        
    def on_epoch_end(self,batch, logs={}):
        self.test_predictions.append(
            self.model.predict_generator(
                DataGenerator(self.test_df.index, None, self.batch_size, self.input_size, self.test_images_dir), verbose=2)[:len(self.test_df)])
        
        # Commented out to save time
        self.valid_predictions.append(
             self.model.predict_generator(
                 DataGenerator(self.valid_df.index, None, self.batch_size, self.input_size, self.valid_images_dir), verbose=2)[:len(self.valid_df)])
        
        print("validation loss: %.4f" %
              weighted_log_loss_metric(self.valid_df.values, 
                                  np.average(self.valid_predictions, axis=0, 
                                              weights=[2**i for i in range(len(self.valid_predictions))])))
        
        print("Test loss: %.4f" %
              weighted_log_loss_metric(self.test_df.values, 
                                  np.average(self.test_predictions, axis=0, 
                                              weights=[2**i for i in range(len(self.test_predictions))])))
        
        # here you could also save the predictions with np.save()


In [7]:
import keras
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input

In [8]:
from keras import backend as K

def weighted_log_loss(y_true, y_pred):
    """
    Can be used as the loss function in model.compile()
    ---------------------------------------------------
    """
    
    class_weights = np.array([2., 1., 1., 1., 1., 1.])
    
    eps = K.epsilon()
    
    y_pred = K.clip(y_pred, eps, 1.0-eps)

    out = -(         y_true  * K.log(      y_pred) * class_weights
            + (1.0 - y_true) * K.log(1.0 - y_pred) * class_weights)
    
    return K.mean(out, axis=-1)
def _normalized_weighted_average(arr, weights=None):
    """
    A simple Keras implementation that mimics that of 
    numpy.average(), specifically for this competition
    """
    
    if weights is not None:
        scl = K.sum(weights)
        weights = K.expand_dims(weights, axis=1)
        return K.sum(K.dot(arr, weights), axis=1) / scl
    return K.mean(arr, axis=1)

def weighted_loss(y_true, y_pred):
    """
    Will be used as the metric in model.compile()
    ---------------------------------------------
    
    Similar to the custom loss function 'weighted_log_loss()' above
    but with normalized weights, which should be very similar 
    to the official competition metric:
        https://www.kaggle.com/kambarakun/lb-probe-weights-n-of-positives-scoring
    and hence:
        sklearn.metrics.log_loss with sample weights
    """
    
    class_weights = K.variable([2., 1., 1., 1., 1., 1.])
    
    eps = K.epsilon()
    
    y_pred = K.clip(y_pred, eps, 1.0-eps)

    loss = -(        y_true  * K.log(      y_pred)
            + (1.0 - y_true) * K.log(1.0 - y_pred))
    
    loss_samples = _normalized_weighted_average(loss, class_weights)
    
    return K.mean(loss_samples)

def weighted_log_loss_metric(trues, preds):
    """
    Will be used to calculate the log loss 
    of the validation set in PredictionCheckpoint()
    ------------------------------------------
    """
    class_weights = [2., 1., 1., 1., 1., 1.]
    
    epsilon = 1e-7
    
    preds = np.clip(preds, epsilon, 1-epsilon)
    loss = trues * np.log(preds) + (1 - trues) * np.log(1 - preds)
    loss_samples = np.average(loss, axis=1, weights=class_weights)

    return - loss_samples.mean()

In [9]:
cnn_model = Sequential()
# Convolutional Layer 1:
cnn_model.add(Conv2D(32, kernel_size = (3, 3), activation = 'linear', input_shape = (128, 128, 3), padding = 'same'))
cnn_model.add(LeakyReLU(alpha = 0.1))
# Max Pooling Layer 1:
cnn_model.add(MaxPooling2D((2, 2)))
# Convolutional Layer 2:
cnn_model.add(Conv2D(64, kernel_size = (3, 3), activation = 'linear', padding = 'same'))
cnn_model.add(LeakyReLU(alpha = 0.1))
# Max Pooling Layer 2:
cnn_model.add(MaxPooling2D((2, 2)))
# Convolutional Layer 3:
cnn_model.add(Conv2D(128, kernel_size = (3, 3), activation = 'linear', padding = 'same'))
cnn_model.add(LeakyReLU(alpha = 0.1))
# Max Pooling Layer 3:
cnn_model.add(MaxPooling2D((2, 2)))
# Flatten Preprocessing:
cnn_model.add(Flatten())
# Fully Connected Layer (Dense Layer):
cnn_model.add(Dense(128, activation = 'linear'))
cnn_model.add(LeakyReLU(alpha = 0.1))
# Output Layer (Dense Layer):
cnn_model.add(Dense(6, activation = 'sigmoid')) # choose one of them? # softmax

# Compile CNN model
cnn_model.compile(loss = "binary_crossentropy", optimizer = keras.optimizers.Adam(), metrics = [weighted_loss])
cnn_model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128, 128, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 64, 64, 64)        0         
_________________________________________________________________
max_poolin

In [10]:
class DataGenerator(keras.utils.Sequence):

    def __init__(self, list_IDs, labels=None, batch_size=1, img_size=(512, 512, 1), 
                 img_dir=train_images_dir, *args, **kwargs):

        self.list_IDs = list_IDs
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.img_dir = img_dir
        self.on_epoch_end()

    def __len__(self):
        return int(ceil(len(self.indices) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indices]
        
        if self.labels is not None:
            X, Y = self.__data_generation(list_IDs_temp)
            return X, Y
        else:
            X = self.__data_generation(list_IDs_temp)
            return X
        
    def on_epoch_end(self):
        
        
        if self.labels is not None: # for training phase we undersample and shuffle
            # keep probability of any=0 and any=1
            keep_prob = self.labels.iloc[:, 0].map({0: 0.35, 1: 0.5})
            keep = (keep_prob > np.random.rand(len(keep_prob)))
            self.indices = np.arange(len(self.list_IDs))[keep]
            np.random.shuffle(self.indices)
        else:
            self.indices = np.arange(len(self.list_IDs))

    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, *self.img_size))
        
        if self.labels is not None: # training phase
            Y = np.empty((self.batch_size, 6), dtype=np.float32)
        
            for i, ID in enumerate(list_IDs_temp):
                #X[i,] = _read(self.img_dir+ID+".dcm", self.img_size)
                X[i,] = [x[1] for x in l_train if x[0] == ID][0]
                Y[i,] = self.labels.loc[ID].values
        
            return X, Y
        
        else: # test phase
            if self.img_dir == '/home/sichun1247/Sampled_Test/':
                for i, ID in enumerate(list_IDs_temp):
                    #X[i,] = _read(self.img_dir+ID+".dcm", self.img_size)
                    X[i,] = [x[1] for x in l_test if x[0] == ID][0]
            else:
                for i, ID in enumerate(list_IDs_temp):
                    #X[i,] = _read(self.img_dir+ID+".dcm", self.img_size)
                    X[i,] = [x[1] for x in l_train if x[0] == ID][0]
            
            return X

In [11]:
from keras.callbacks import EarlyStopping
from sklearn.model_selection import ShuffleSplit
#datagen = ImageDataGenerator(
#    rotation_range = 30,
#    horizontal_flip = True)
#datagen.fit(train_X)
test_df = test_df
df = train_df

#early_stopping = EarlyStopping(monitor='val_weighted_loss', patience=2)
ss = ShuffleSplit(n_splits=10, test_size=0.1, random_state=42).split(df.index)
train_idx, valid_idx = next(ss)

pred_history = PredictionCheckpoint(test_df, df.iloc[valid_idx], input_size=(128, 128, 3))

# Fits the model on batches with real-time data augmentation:
#cnn_model_process = cnn_model.fit_generator(datagen.flow(train_X, train_Y, batch_size = 32),
#                                                             steps_per_epoch = len(train_X) / 32, 
#                                                             epochs = 20, verbose = 1, 
#                                                             validation_data = (test_X, test_Y))

#cnn_model_process = cnn_model.fit(train_X, train_Y, batch_size = 32, epochs = 30, verbose = 1, 
#                                  validation_data = (test_X, test_Y))

cnn_model_process = cnn_model.fit_generator(
                                    DataGenerator(
                                        df.iloc[train_idx].index, 
                                        df.iloc[train_idx], 
                                        batch_size = 32, 
                                        img_size=(128, 128, 3), 
                                        img_dir=train_images_dir
                                    ),
                                    steps_per_epoch = len(df.iloc[train_idx]) / 32,
                                    epochs=20,
                                    verbose=1,
                                    callbacks=[pred_history]
                                )


Epoch 1/20
3973/3972 [==============================] - 2816s 709ms/step - loss: 0.3001 - weighted_loss: 0.3138
validation loss: 0.2731
Test loss: 0.2915
Epoch 2/20
3973/3972 [==============================] - 2809s 707ms/step - loss: 0.2194 - weighted_loss: 0.2303
validation loss: 0.2353
Test loss: 0.2562
Epoch 3/20
3973/3972 [==============================] - 2786s 701ms/step - loss: 0.1616 - weighted_loss: 0.1722
validation loss: 0.2126
Test loss: 0.2378
Epoch 4/20
3973/3972 [==============================] - 2791s 703ms/step - loss: 0.1162 - weighted_loss: 0.1257
validation loss: 0.1982
Test loss: 0.2247
Epoch 5/20
3973/3972 [==============================] - 2784s 701ms/step - loss: 0.0883 - weighted_loss: 0.0964
validation loss: 0.1973
Test loss: 0.2253
Epoch 6/20
3973/3972 [==============================] - 2791s 702ms/step - loss: 0.0659 - weighted_loss: 0.0724
validation loss: 0.1993
Test loss: 0.2312
Epoch 7/20
 293/3972 [=>............................] - ETA: 43:16 - loss: 

KeyboardInterrupt: 

In [ ]:
# cnn_model_process.history

In [ ]:
from matplotlib import pyplot

pyplot.plot(cnn_model_process.history['weighted_loss'], marker = 'o', label='train')
pyplot.plot(cnn_model_process.history['val_weighted_loss'], marker = 'o', label='validation')
pyplot.xlabel('Epochs', fontsize = 14)
pyplot.ylabel('Weighted Loss',fontsize = 14)
pyplot.title('CNN Weighted Loss Trainig VS Testing', fontsize = 14)
pyplot.legend()
pyplot.show()

In [ ]:

cnn_model_dropout = Sequential()
# Convolutional Layer 1:
cnn_model_dropout.add(Conv2D(32, kernel_size = (3, 3), activation = 'linear', input_shape = (128, 128, 3), padding = 'same'))
cnn_model_dropout.add(LeakyReLU(alpha = 0.1))
# Max Pooling Layer 1:
cnn_model_dropout.add(MaxPooling2D((2, 2)))
# Convolutional Layer 2:
cnn_model_dropout.add(Conv2D(64, kernel_size = (3, 3), activation = 'linear', padding = 'same'))
cnn_model_dropout.add(LeakyReLU(alpha = 0.1))
# Max Pooling Layer 2:
cnn_model_dropout.add(MaxPooling2D((2, 2)))
# Convolutional Layer 3:
cnn_model_dropout.add(Conv2D(128, kernel_size = (3, 3), activation = 'linear', padding = 'same'))
cnn_model_dropout.add(LeakyReLU(alpha = 0.1))
# Max Pooling Layer 3:
cnn_model_dropout.add(MaxPooling2D((2, 2)))
# Flatten Preprocessing:
cnn_model_dropout.add(Flatten())
# Fully Connected Layer (Dense Layer):
cnn_model_dropout.add(Dense(128, activation = 'linear'))
cnn_model_dropout.add(BatchNormalization())
cnn_model_dropout.add(LeakyReLU(alpha = 0.1))

# Dropout Layer 4:
cnn_model_dropout.add(Dropout(0.3))
# Output Layer (Dense Layer):
cnn_model_dropout.add(Dense(6, activation = 'sigmoid'))

# Compile CNN model
cnn_model_dropout.compile(loss = "binary_crossentropy", optimizer = keras.optimizers.Adam(), metrics = [weighted_loss])
cnn_model_dropout.summary()

In [ ]:
# Fits the model on batches with real-time data augmentation:
# cnn_model_dropout_process = cnn_model_dropout.fit_generator(datagen.flow(train_X, train_Y, batch_size = 32),
#                                                             steps_per_epoch = len(train_X) / 32, 
#                                                             epochs = 20, verbose = 1, 
#                                                             validation_data = (test_X, test_Y))

cnn_model_dropout_process = cnn_model_dropout.fit(train_X, train_Y, batch_size = 32, epochs = 20, verbose = 1, 
                                                  validation_data = (test_X, test_Y))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize = (10, 5))
axes[0].plot(range(1, len(cnn_model_process.history['loss']) + 1), cnn_model_process.history['loss'], linestyle = 'solid', marker = 'o', color = 'crimson', label = 'Training Loss')
axes[0].plot(range(1, len(cnn_model_process.history['val_loss']) + 1), cnn_model_process.history['val_loss'], linestyle = 'solid', marker = 'o', color = 'dodgerblue', label = 'Testing Loss')
axes[0].set_xlabel('Epochs', fontsize = 14)
axes[0].set_ylabel('Loss',fontsize = 14)
axes[0].set_title('CNN Loss Trainig VS Testing', fontsize = 14)
axes[0].legend(loc = 'best')
axes[1].plot(range(1, len(cnn_model_process.history['weighted_loss']) + 1), cnn_model_process.history['weighted_loss'], linestyle = 'solid', marker = 'o', color = 'crimson', label = 'Training Weighted Loss')
axes[1].plot(range(1, len(cnn_model_process.history['val_weighted_loss']) + 1), cnn_model_process.history['val_weighted_loss'], linestyle = 'solid', marker = 'o', color = 'dodgerblue', label = 'Testing Weighted Loss')
axes[1].set_xlabel('Epochs', fontsize = 14)
axes[1].set_ylabel('Weighted Loss',fontsize = 14)
axes[1].set_title('CNN Weighted Loss Trainig VS Testing', fontsize = 14)
axes[1].legend(loc = 'best')

fig, axes = plt.subplots(1, 2, figsize = (10, 5))
axes[0].plot(range(1, len(cnn_model_dropout_process.history['loss']) + 1), cnn_model_dropout_process.history['loss'], linestyle = 'solid', marker = 'o', color = 'crimson', label = 'Training Accuracy')
axes[0].plot(range(1, len(cnn_model_dropout_process.history['val_loss']) + 1), cnn_model_dropout_process.history['val_loss'], linestyle = 'solid', marker = 'o', color = 'dodgerblue', label = 'Testing Accuracy')
axes[0].set_xlabel('Epochs', fontsize = 14)
axes[0].set_ylabel('Loss',fontsize = 14)
axes[0].set_title('CNN Dropout Loss Trainig VS Testing', fontsize = 14)
axes[0].legend(loc = 'best')
axes[1].plot(range(1, len(cnn_model_dropout_process.history['weighted_loss']) + 1), cnn_model_dropout_process.history['weighted_loss'], linestyle = 'solid', marker = 'o', color = 'crimson', label = 'Training Loss')
axes[1].plot(range(1, len(cnn_model_dropout_process.history['val_weighted_loss']) + 1), cnn_model_dropout_process.history['val_weighted_loss'], linestyle = 'solid', marker = 'o', color = 'dodgerblue', label = 'Testing Loss')
axes[1].set_xlabel('Epochs', fontsize = 14)
axes[1].set_ylabel('Weighted Loss',fontsize = 14)
axes[1].set_title('CNN Dropout Weighted Loss Trainig VS Testing', fontsize = 14)
axes[1].legend(loc = 'best')

In [ ]:
# class DataGenerator(keras.utils.Sequence):

#     def __init__(self, list_IDs, labels=None, batch_size=1, img_size=(512, 512, 1), 
#                  img_dir=train_images_dir, *args, **kwargs):

#         self.list_IDs = list_IDs
#         self.labels = labels
#         self.batch_size = batch_size
#         self.img_size = img_size
#         self.img_dir = img_dir
#         self.on_epoch_end()

#     def __len__(self):
#         return int(ceil(len(self.indices) / self.batch_size))
#     def __getitem__(self, index):
#         indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
#         list_IDs_temp = [self.list_IDs[k] for k in indices]
        
#         if self.labels is not None:
#             X, Y = self.__data_generation(list_IDs_temp)
#             return X, Y
#         else:
#             X = self.__data_generation(list_IDs_temp)
#             return X
#     def on_epoch_end(self):
        
        
#         if self.labels is not None: # for training phase we undersample and shuffle
#             # keep probability of any=0 and any=1
#             keep_prob = self.labels.iloc[:, 0].map({0: 0.35, 1: 0.5})
#             keep = (keep_prob > np.random.rand(len(keep_prob)))
#             self.indices = np.arange(len(self.list_IDs))[keep]
#             np.random.shuffle(self.indices)
#         else:
#             self.indices = np.arange(len(self.list_IDs))
#     def __data_generation(self, list_IDs_temp):
#         X = np.empty((self.batch_size, *self.img_size))
        
#         if self.labels is not None: # training phase
#             Y = np.empty((self.batch_size, 6), dtype=np.float32)
        
#             for i, ID in enumerate(list_IDs_temp):
#                 X[i,] = _read(self.img_dir+ID+".dcm", self.img_size)
#                 Y[i,] = self.labels.loc[ID].values
        
#             return X, Y
        
#         else: # test phase
#             for i, ID in enumerate(list_IDs_temp):
#                 X[i,] = _read(self.img_dir+ID+".dcm", self.img_size)
            
#             return X


In [ ]:
# from keras import backend as K

# def weighted_log_loss(y_true, y_pred):
#     """
#     Can be used as the loss function in model.compile()
#     ---------------------------------------------------
#     """
    
#     class_weights = np.array([2., 1., 1., 1., 1., 1.])
    
#     eps = K.epsilon()
    
#     y_pred = K.clip(y_pred, eps, 1.0-eps)

#     out = -(         y_true  * K.log(      y_pred) * class_weights
#             + (1.0 - y_true) * K.log(1.0 - y_pred) * class_weights)
    
#     return K.mean(out, axis=-1)
# def _normalized_weighted_average(arr, weights=None):
#     """
#     A simple Keras implementation that mimics that of 
#     numpy.average(), specifically for this competition
#     """
    
#     if weights is not None:
#         scl = K.sum(weights)
#         weights = K.expand_dims(weights, axis=1)
#         return K.sum(K.dot(arr, weights), axis=1) / scl
#     return K.mean(arr, axis=1)

# def weighted_loss(y_true, y_pred):
#     """
#     Will be used as the metric in model.compile()
#     ---------------------------------------------
    
#     Similar to the custom loss function 'weighted_log_loss()' above
#     but with normalized weights, which should be very similar 
#     to the official competition metric:
#         https://www.kaggle.com/kambarakun/lb-probe-weights-n-of-positives-scoring
#     and hence:
#         sklearn.metrics.log_loss with sample weights
#     """
    
#     class_weights = K.variable([2., 1., 1., 1., 1., 1.])
    
#     eps = K.epsilon()
    
#     y_pred = K.clip(y_pred, eps, 1.0-eps)

#     loss = -(        y_true  * K.log(      y_pred)
#             + (1.0 - y_true) * K.log(1.0 - y_pred))
    
#     loss_samples = _normalized_weighted_average(loss, class_weights)
    
#     return K.mean(loss_samples)

# def weighted_log_loss_metric(trues, preds):
#     """
#     Will be used to calculate the log loss 
#     of the validation set in PredictionCheckpoint()
#     ------------------------------------------
#     """
#     class_weights = [2., 1., 1., 1., 1., 1.]
    
#     epsilon = 1e-7
    
#     preds = np.clip(preds, epsilon, 1-epsilon)
#     loss = trues * np.log(preds) + (1 - trues) * np.log(1 - preds)
#     loss_samples = np.average(loss, axis=1, weights=class_weights)

#     return - loss_samples.mean()